In [1]:
from numpy import *
from scipy.sparse import *
from scipy.sparse.linalg import *
from scipy.stats import *
from sklearn.preprocessing import *

In [2]:
monoArtist=load("target_playlist_mono_artist.npy")
monoArtistPlID=monoArtist[:,0]
monoArtistArtistID=monoArtist[:,1]
artistIndexes=load("uniqueArtists_NeededToIndexThe_ArtistReducedMatrices.npy")
artistTracks=load_npz("artists_with_tracksID_ordered_by_occurrencies.npz")
target_playlists=genfromtxt("target_playlists.csv",skip_header=1)
playlists_with_tracks=load("playlists_with_tracks.npy")
target_tracks_ordered = load("targetTracksOrdered.npy")


def getsimil(plid, similrow, n):
    ptracks = playlists_with_tracks[where(playlists_with_tracks[:,0]==plid),1:]
    maxi = flip(argsort(similrow), axis=0)
    r = []
    for m in maxi:
        if(not isin(target_tracks_ordered[m], ptracks)):
           r.append(target_tracks_ordered[m])
           if(len(r)==n):
               return r
        
    PLIDs = load("fucking_ordered_playlist.npy")
    TIDs = load("fucking_ordered_tracks.npy")

In [36]:
import sys
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush() 

tar=10000

def computeMono(fname, finalPlSim):
    targetP = load("fucking_ordered_playlist.npy")
    playlists_with_tracks=load("playlists_with_tracks.npy")
    target_playlists=genfromtxt("target_playlists.csv",skip_header=1)
    targetP = targetP[isin(targetP, target_playlists)]
    with open(fname,"a") as myfile:
        myfile.write("playlist_id,track_ids\n")
        for pl, similsum, i in zip(targetP, finalPlSim, range(tar)):
            progress(i, tar, status="done")
            plID=pl
            s = str(int(plID))
            s += ","
            if(isin(plID,monoArtistPlID)):#guarda se c'è un artista solo
                artist=monoArtistArtistID[where(monoArtistPlID==plID)[0][0]]#l'unico artista della playlist
                artistIndex=where(artistIndexes==artist)[0][0]#cerca la sua posizione nella matrice delle tracce
                thisArtistTracks=artistTracks.getrow(artistIndex).data#prende le sue tracce
                rr=array([t for t in thisArtistTracks if t not in playlists_with_tracks[where(playlists_with_tracks[:,0]==plID),1:] and isin(t, target_tracks_ordered)])#prende tutte quelle non già presenti

                if(len(rr)>=5):
                    r = array(rr).take(range(5))
                else:
                    r4 = getsimil(pl, similsum.data, 5)
                    r4 = array([el for el in r4 if el not in rr])
                    r = concatenate((rr, r4.take(range(5-len(rr)))))
            else:
                r = getsimil(pl, similsum.data, 5)#non c'è un artista solo quindi guarda solo i tag
            for el in r:
                s+=str(el)
                s+=" "
            myfile.write(s + "\n")
            
def compute(fname, finalPlSim):
    targetP = load("fucking_ordered_playlist.npy")
    playlists_with_tracks=load("playlists_with_tracks.npy")
    target_playlists=genfromtxt("target_playlists.csv",skip_header=1)
    targetP = targetP[isin(targetP, target_playlists)]
    with open(fname,"a") as myfile:
        myfile.write("playlist_id,track_ids\n")
        for pl, similsum, i in zip(targetP, finalPlSim, range(tar)):
            progress(i, tar, status="done")
            plID=pl
            s = str(int(plID))
            s += ","
            r = getsimil(pl, similsum.data, 5)#non c'è un artista solo quindi guarda solo i tag
            for el in r:
                s+=str(el)
                s+=" "
            myfile.write(s + "\n")            
            

In [4]:
PxA = load_npz("fucking_ordered_playlist_artist.npz")
PxT = load_npz("all_playlist_with_tracks_URM.npz")
TxA = load_npz("fucking_ordered_track_artist.npz")


In [12]:
similPAT = normalize(PxA) * TxA.T #S1, similarity tra playlist artist e track artist

In [13]:
similPAT

<45649x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 21113789 stored elements in Compressed Sparse Row format>

In [33]:
def filterTarget(m):
    PLIDs = load("fucking_ordered_playlist.npy")
    TIDs = load("fucking_ordered_tracks.npy")
    target_playlists=genfromtxt("target_playlists.csv",skip_header=1)
    target_tracks=genfromtxt("target_tracks.csv", skip_header=1)
    k = m[isin(PLIDs, target_playlists)]
    k = k[:, isin(TIDs, target_tracks)]
    return k

def filterPLrow(m):
    PLIDs = load("fucking_ordered_playlist.npy")
    target_playlists=genfromtxt("target_playlists.csv",skip_header=1)
    k = m[isin(PLIDs, target_playlists)]
    return k 

def filterTcol(m):
    TIDs = load("fucking_ordered_tracks.npy")
    target_tracks=genfromtxt("target_tracks.csv", skip_header=1)
    k = m[:, isin(TIDs, target_tracks)]
    return k

In [14]:
S1 = filterTarget(similPAT)

In [15]:
trackAlbum=load_npz("fucking_ordered_track_album.npz")

In [16]:
TTAL = trackAlbum * trackAlbum.T
TTAL.setdiag(0)

C:\Users\Andrea\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [17]:
S2 = PxT * TTAL  #S2: 2 canzoni sono simili se hanno lo stesso album, playlist è simile alla canzone se la contiene

In [18]:
S2 = filterTarget(S2)

In [19]:
S2 = normalize(S2)

In [38]:
S = S1+1.5*S2    #a questo punto andrebbe bene, ma in realtà alcune canzoni hanno meno di 5 similarità non zero
                #S3 è SBAGLIATA. UN'ALTRA IDEA SAREBBE METTERE IN SOTTOFONDO LA VECCHIA TFIDF SUI TAG CON UN PESO BASSISSIMO, 
               #MA CHE ALMENO CI EVITA L'ERRORE.

In [5]:
S3 = PxA * PxA.T          #S3: 2 playlist sono simili se contengono gli stessi artisti, moltiplicato poi per la matrice con le tracks

In [6]:
S3 = normalize(S3)
S3.setdiag(0)

In [9]:
S3 = filterPLrow(S3.T)

In [11]:
S3 = S3 * PxT

In [34]:
S3 = filterTcol(S3)

In [39]:
S = S + S3   

In [40]:
compute("today2.csv", S)